In [75]:
import json

import gpflow
import matplotlib.pyplot as plt
import numpy as np
from hecstac.ras.item import RASModelItem
from sklearn.model_selection import KFold

from gpras.gpr import RasEmulator


In [235]:
# Initialize GPR model
import importlib

import gpras.gpr

importlib.reload(gpras.gpr)

with open(emulator_meta_path) as f:
    emulator_meta = json.load(f)
gpr = RasEmulator(**emulator_meta)
import logging

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)


In [3]:
# Constants
ras_path = "/workspaces/gpras/data_dir/Muncie/Muncie.prj"
crs = "ESRI:102673"
stac_path = "/workspaces/gpras/data_dir/Muncie/Muncie.stac.json"
emulator_meta_path = "ras_emulator.json"

In [4]:
# Make emulator metadata json.  Only need to run once.
metadata = {
    "ras_model_path": stac_path,
    "hf_mesh_id": "2D Interior Area",
    "lf_mesh_id": "2D Interior Area",
    "lf_runs": ["lf1", "lf2", "lf3"],
    "hf_runs": ["hf1", "hf2", "hf3"],
    "testing_runs": [],
    "enable_cell_weighting": True
}
with open(emulator_meta_path, mode="w") as f:
    json.dump(metadata, f, indent=4)

In [5]:
# Make initial STAC item. Only need to run once.
stac = RASModelItem.from_prj(ras_path, crs)
with open(stac_path, mode="w") as f:
    json.dump(stac.to_dict(), f, indent=4)

Ras model has no current plan


Classify HF model cells to identify Wet, Always Dry (AD), Temporarily Flooded (TF) and Always Flooded (AF) cells.
Create shapefile to visualize and review classification

In [68]:
classification_qc_shp = "/workspaces/gpras/data_dir/Muncie_qc/classification_qc.shp"
gpr.flood_classification_shp(gpr.hf_runs, classification_qc_shp)

2025-06-26 07:53:40,579 - INFO - Created 30,504 records


Create training data for K-fold validation and extract spatial EOFs for each HF data training folds

In [229]:
gpr.perform_eof_kfold_analysis(n_splits=3, n_modes=100)

2025-06-26 14:24:50,956 - INFO - Starting EOF analysis across 3 folds with 100 EOF modes...
2025-06-26 14:24:50,970 - INFO - 
--- Fold fold_0 ---
2025-06-26 14:24:50,971 - INFO - Training runs: ['hf2', 'hf3']


2025-06-26 14:24:51,813 - INFO - Unioned wet cells: 4815
2025-06-26 14:24:51,849 - INFO - Computed weighted temporal mean over wet cells
2025-06-26 14:24:53,070 - INFO - Partial fit on run hf2 (289 timesteps)
2025-06-26 14:24:55,469 - INFO - Partial fit on run hf3 (289 timesteps)
2025-06-26 14:24:55,470 - INFO - Selected 16 significant EOFs (North's Rule)
2025-06-26 14:24:55,471 - INFO - EOFs and statistics stored for fold fold_0

2025-06-26 14:24:55,471 - INFO - 
--- Fold fold_1 ---
2025-06-26 14:24:55,471 - INFO - Training runs: ['hf1', 'hf3']
2025-06-26 14:24:55,473 - INFO - Unioned wet cells: 4885
2025-06-26 14:24:55,480 - INFO - Computed weighted temporal mean over wet cells
2025-06-26 14:24:56,892 - INFO - Partial fit on run hf1 (289 timesteps)
2025-06-26 14:24:58,949 - INFO - Partial fit on run hf3 (289 timesteps)
2025-06-26 14:24:58,950 - INFO - Selected 17 significant EOFs (North's Rule)
2025-06-26 14:24:58,951 - INFO - EOFs and statistics stored for fold fold_1

2025-06-26 14

Visualize EOF results

In [230]:
# Save all plots into a folder
gpr.visualize_all_norths_rule(output_dir="/workspaces/gpras/production/plots/norths_rule")
gpr.visualize_all_kfold_eof_results_separate(output_dir="/workspaces/gpras/production/plots/eof_modes", n_modes_to_plot=3)



Visualizing results for fold_0...


2025-06-26 14:25:24,819 - DEBUG - locator: <matplotlib.ticker.AutoLocator object at 0x7f448d11f450>
2025-06-26 14:25:25,016 - DEBUG - locator: <matplotlib.ticker.AutoLocator object at 0x7f448cf53790>
2025-06-26 14:25:25,237 - DEBUG - locator: <matplotlib.ticker.AutoLocator object at 0x7f448c5d97d0>
2025-06-26 14:25:25,583 - DEBUG - locator: <matplotlib.ticker.AutoLocator object at 0x7f448b2bc8d0>



Visualizing results for fold_1...


2025-06-26 14:25:25,766 - DEBUG - locator: <matplotlib.ticker.AutoLocator object at 0x7f448d2a3790>
2025-06-26 14:25:25,957 - DEBUG - locator: <matplotlib.ticker.AutoLocator object at 0x7f449d720c10>
2025-06-26 14:25:26,278 - DEBUG - locator: <matplotlib.ticker.AutoLocator object at 0x7f4489bc92d0>



Visualizing results for fold_2...


2025-06-26 14:25:26,462 - DEBUG - locator: <matplotlib.ticker.AutoLocator object at 0x7f448ce94910>
2025-06-26 14:25:26,681 - DEBUG - locator: <matplotlib.ticker.AutoLocator object at 0x7f448d168c10>


Visualize High Fidelity ECs 

In [231]:
gpr.data_y_train
gpr.visualize_all_kfold_ecs(n_modes_to_plot=3, output_dir="/workspaces/gpras/production/plots/hf_ecs", show_run_boundaries=True)


2025-06-26 14:25:46,743 - INFO - Computing and caching HF ECs as training targets (data_y_train)
2025-06-26 14:25:46,744 - INFO - Deriving ECs for fold: fold_0
2025-06-26 14:25:46,746 - INFO - Centering and weighting WSE data for EC computation
2025-06-26 14:25:46,784 - INFO - Derived ECs for run: hf2, shape: (289, 16)
2025-06-26 14:25:46,788 - INFO - Centering and weighting WSE data for EC computation
2025-06-26 14:25:46,802 - INFO - Derived ECs for run: hf3, shape: (289, 16)
2025-06-26 14:25:46,804 - INFO - Stored HF ECs for fold_0 into self.Y_train
2025-06-26 14:25:46,805 - INFO - Deriving ECs for fold: fold_1
2025-06-26 14:25:46,811 - INFO - Centering and weighting WSE data for EC computation
2025-06-26 14:25:46,836 - INFO - Derived ECs for run: hf1, shape: (289, 17)
2025-06-26 14:25:46,845 - INFO - Centering and weighting WSE data for EC computation
2025-06-26 14:25:46,852 - INFO - Derived ECs for run: hf3, shape: (289, 17)
2025-06-26 14:25:46,853 - INFO - Stored HF ECs for fold_1

Create Low-Fidelity ECs

In [232]:
gpr.data_x
gpr.data_x_train

/workspaces/gpras/gpras/gpr.py:856: UserWarning: `keep_geom_type=True` in overlay resulted in 85 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
  
2025-06-26 14:26:17,115 - INFO - Computing and caching LF ECs as training inputs (data_x_train)
2025-06-26 14:26:17,115 - INFO - Deriving LF ECs using HF EOFs for fold_0
2025-06-26 14:26:17,124 - INFO - Centering and weighting WSE data for EC computation
2025-06-26 14:26:17,141 - INFO - LF ECs derived for run lf2 with shape (289, 16)
2025-06-26 14:26:17,163 - INFO - Centering and weighting WSE data for EC computation
2025-06-26 14:26:17,181 - INFO - LF ECs derived for run lf3 with shape (289, 16)
2025-06-26 14:26:17,182 - INFO - Stored LF ECs for fold_0 into self.X_train
2025-06-26 14:26:17,187 - INFO - Deriving LF ECs using HF EOFs for fold_1
2025-06-26 14:26:17,201 - INFO - Centering and weighting WSE data for EC computation
2025-06-26 14:26:17,211 - INFO - LF ECs derived fo

lf2
289
lf3
289
lf1
289
lf3
289
lf1
289
lf2
289


{'fold_0': array([[-52023.23825454, -14416.68075227,  22952.64837712, ...,
           1621.25758958,   1240.4320369 ,  -1343.77127433],
        [-52023.23825454, -14416.68075227,  22952.64837712, ...,
           1621.25758958,   1240.4320369 ,  -1343.77127433],
        [-52023.23825454, -14416.68075227,  22952.64837712, ...,
           1621.25758958,   1240.4320369 ,  -1343.77127433],
        ...,
        [-51411.45798001, -14407.77560122,  22922.68565733, ...,
           1620.95512249,   1240.09762565,  -1343.41839262],
        [-51411.45268426, -14407.76874984,  22922.68265143, ...,
           1620.95519687,   1240.09741675,  -1343.41778454],
        [-51411.44727305, -14407.7620253 ,  22922.67945884, ...,
           1620.95521954,   1240.09742958,  -1343.41740597]]),
 'fold_1': array([[-52045.41084536, -16818.83455928,  26179.59492987, ...,
           2561.82667604,    600.74318532,    871.57364836],
        [-52045.41084536, -16818.83455928,  26179.59492987, ...,
           2561.82

Visualize LF ECs

In [233]:
gpr.visualize_all_lf_ecs(n_modes_to_plot=3, output_dir="/workspaces/gpras/production/plots/lf_ecs")


2025-06-26 14:26:34,863 - INFO - Visualizing LF ECs for fold_0
2025-06-26 14:26:35,209 - INFO - Saved LF EC plot to /workspaces/gpras/production/plots/lf_ecs/LF_ECs_fold_0.png
2025-06-26 14:26:35,209 - INFO - Visualizing LF ECs for fold_1
2025-06-26 14:26:35,537 - INFO - Saved LF EC plot to /workspaces/gpras/production/plots/lf_ecs/LF_ECs_fold_1.png
2025-06-26 14:26:35,538 - INFO - Visualizing LF ECs for fold_2
2025-06-26 14:26:35,861 - INFO - Saved LF EC plot to /workspaces/gpras/production/plots/lf_ecs/LF_ECs_fold_2.png


In [234]:
gpr.compare_lf_hf_ecs(n_modes_to_plot=3, output_dir="/workspaces/gpras/production/plots/compare_hf_lf_ecs")